In [1]:
import pickle
import xgboost as xgb
import time
import numpy as np
import pandas as pd

/home/naireen/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# load csv model,
# create table of features
# how to pass each individual row and get predictions
# are these logit scores or probabilities?
path_to_csv = "/home/naireen/Documents/DanML/MLstability/csvs/BetaFeats/sifOrbsMore=10000.0Nout=100window=10.csv"
dataset = pd.read_csv(path_to_csv)

features = []
features += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['avg_beta12','std_beta12','min_beta12','max_beta12','avg_beta23','std_beta23','min_beta23','max_beta23']
X = dataset[features]
X.head(5)

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,norm_a2_slope,norm_a3_slope,avg_beta12,std_beta12,min_beta12,max_beta12,avg_beta23,std_beta23,min_beta23,max_beta23
0,0.396282,0.481704,0.000329,0.001083,0.000214,0.000456,0.000701,0.001978,0.000446,0.000964,...,2.144832e-10,2.681898e-11,8.488064,0.048209,8.344623,8.551727,15.791595,0.085995,15.576612,16.019985
1,1.034575,0.823935,0.000739,0.002137,0.000202,0.000463,0.000200,0.000350,0.000226,0.000377,...,3.559377e-11,-9.314702e-11,8.414875,0.039142,8.308582,8.495900,15.314101,0.037065,15.254833,15.368196
2,0.440470,0.491261,0.001951,0.003550,0.001735,0.002860,0.002303,0.004034,0.002080,0.003409,...,-3.330088e-11,-4.095504e-11,3.566625,0.132349,3.331239,3.773868,25.113282,0.137779,24.898424,25.351948
3,0.432098,0.550624,0.000026,0.000120,0.000005,0.000009,0.000019,0.000050,0.000016,0.000041,...,7.816527e-13,-7.257099e-12,18.513867,0.003597,18.498104,18.521611,19.142029,0.008997,19.120348,19.156777
4,5.722053,17.442426,0.028284,0.063028,0.004313,0.010716,0.038832,0.160685,0.090892,0.112168,...,3.913858e-08,-2.610208e-08,23.314674,6.756316,3.957209,38.942908,-19.036068,5.544896,-21.930247,4.156005


In [3]:
data_dump  = open("../Testing/uncompressed_final.txt", "r")
counter = 0
booster_list = []
normal_order = []
file_length = len(data_dump.readlines())

data_dump.seek(0) #reset file read pointer
#make a list of lists, where each inner list is the data for each booster


temp = []
keep_going = True
#while keep_going:
for i in range(file_length): # guess value
    line = data_dump.next().strip()
    if "booster[" in line:
        #save old one, if its not the first time this is run
        if len(temp)!=0:
            #attach an unaliased copy
            booster_list.append(temp[:])
        #begin new array ouput
        temp = []
    else:
        temp.append(line)
#need to manually append the very last one, since there won't be a booster marker after it
booster_list.append(temp[:])

In [4]:
#change encoding, should not be necessary
print(features)

['avg_iH1', 'avg_iH2', 'norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1', 'norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2', 'norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3', 'avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1', 'avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2', 'avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3', 'norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope', 'avg_beta12', 'std_beta12', 'min_beta12', 'max_beta12', 'avg_beta23', 'std_beta23', 'min_beta23', 'max_beta23']


In [5]:
def tree_to_code(file_name, feat_names, save_script, num):
    #saved script is a file object, not a file path

    save_script.write("\n    def tree{}(vals):".format(num))
    saved_code.write("\n        " +", ".join(feat_names) + " = vals")

    def recurse(node, depth):
        
        #each node is a line in the document
        #depth
        indent = "    "* depth
        #print(node)
        #check its not terminal        
        if node.split(":")[1].split("=")[0] != "leaf":
            #split at this value
            #try:
            threshold = node.split("<")[1].split("]")[0]
            #except:
            #feature being examined
            feature_name  = node.split("[")[1].split("<")[0]
            #if yes, go here (right)
            next_line_yes = int(node.split("=")[1].split(",")[0])
            # if no or missing, go here (left)
            next_line_no = int(node.split("=")[2].split(",")[0])
            #print "{}if {} < {}:".format(indent, feature_name, threshold) 
            save_script.write("\n{}if {} < {}:".format(indent, feature_name, threshold))
            #if not a terminal node
            recurse(file_name[next_line_yes], depth +1)
            #print "{}else: #threshold {} > {}".format(indent, feature_name, threshold) 
            save_script.write("\n{}else: #threshold {} > {}".format(indent, feature_name, threshold))
            recurse(file_name[next_line_no] , depth +1)
        else:
            #its terminal
            val = node.split("=")[1]
            #print "{}return {}".format(indent, val)
            save_script.write("\n{}return {}".format(indent, val))
    #start at head of tree, depth of 1, since you need to indent after def... 
    recurse(file_name[0], 2)
    return save_script

In [6]:
saved_code = open('coded_xgb_model_test.py','w+') 

#print("\ndef tree(vals):")
#print("\n    " +", ".join(feat_names) + " = vals")
saved_code.write("import numpy as np")
saved_code.write("\ndef avg_booster_models(vals):")
saved_code.write("\n    values = np.zeros({})".format(len(booster_list)))
for i, booster_tree in enumerate(booster_list):
    saved_code = tree_to_code(booster_tree, features, saved_code, i)
    saved_code.write("\n    values[{0}] = tree{0}(vals)".format(i))
saved_code.write("\n    avg_val = np.sum(values)")
saved_code.write("\n    return avg_val, values")
saved_code.close()
#check line 346

In [7]:
import coded_xgb_model_test

In [16]:
#print(X.loc[1].values)
max_val = 0
min_val = 1
for i in range(9):
    val = coded_xgb_model_test.avg_booster_models(X.loc[i].values)
    print(val[0], val[1])

(0.3308147, array([-0.170113 ,  0.0338133,  0.0869398, -0.0226714,  0.224358 ,
       -0.156022 , -0.139861 ,  0.216012 ,  0.136377 ,  0.121982 ]))
(0.6136694, array([-0.170113 ,  0.0338133,  0.0857531,  0.26137  ,  0.224358 ,
       -0.156022 , -0.139861 ,  0.216012 ,  0.136377 ,  0.121982 ]))
(0.5121697, array([-0.170113 ,  0.209864 , -0.177662 ,  0.208937 ,  0.169478 ,
        0.238509 , -0.139861 , -0.0734764,  0.187557 ,  0.0589371]))
(0.24215239999999996, array([ 0.0266125,  0.0338133,  0.278104 ,  0.0946914, -0.315051 ,
       -0.25299  ,  0.229437 ,  0.216477 , -0.0554273, -0.0135145]))
(0.7884689, array([-0.0107403, -0.104382 , -0.177662 ,  0.221611 ,  0.133931 ,
        0.238509 ,  0.15315  ,  0.149265 ,  0.208326 , -0.0235388]))
(0.9286403, array([ 0.123982 ,  0.0338133,  0.290249 ,  0.26137  ,  0.224358 ,
       -0.156022 ,  0.229437 ,  0.120642 , -0.321171 ,  0.121982 ]))
(1.1182251, array([ 0.313819 , -0.104382 , -0.0305391,  0.266804 ,  0.133931 ,
        0.238509 ,  0.1